In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('train_data.xlsx')

In [4]:
df.dropna(inplace=True)
df.drop(['Day Summary','Precipitation Type'],axis = 1, inplace = True)

In [23]:
import datetime as dt
df['Date'] = pd.to_datetime(df['Date'],utc=True)
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['hour'] = df['Date'].dt.hour

In [24]:
from sklearn.model_selection import train_test_split
dff = df.drop(['Date','Temperature (in C)'], axis = 1)
y = df['Temperature (in C)']

In [25]:
x_train,x_test,y_train,y_test = train_test_split(dff,y,test_size = 0.3,random_state = 42)

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [28]:
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1))
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))

In [29]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [31]:
model = Sequential()
model.add(SimpleRNN(units=128, input_shape=(None,1), activation="relu"))
model.add(Dense(8, activation="relu")) 
model.add(Dense(10,activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='rmsprop')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 128)               16640     
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 1032      
_________________________________________________________________
dense_16 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_17 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_18 (Dense)             (None, 10)                60        
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 11        
Total params: 17,888
Trainable params: 17,888
Non-trainable params: 0
__________________________________________________

In [32]:
history = model.fit(x_train,y_train, epochs=5, batch_size=64,validation_data=(x_test,y_test))

Train on 55899 samples, validate on 23958 samples
Epoch 1/5
55899/55899 [==============================] - 8s 144us/sample - loss: 2.4246 - val_loss: 0.8110
Epoch 2/5
55899/55899 [==============================] - 7s 120us/sample - loss: 1.5381 - val_loss: 0.3844
Epoch 3/5
55899/55899 [==============================] - 7s 119us/sample - loss: 1.3135 - val_loss: 1.3107
Epoch 4/5
55899/55899 [==============================] - 7s 123us/sample - loss: 1.1713 - val_loss: 1.1035
Epoch 5/5
55899/55899 [==============================] - 7s 126us/sample - loss: 1.0731 - val_loss: 1.2976


In [33]:
dff.columns

Index(['Apparent Temperature (in C)', 'Humidity', 'Wind Speed (in km/h)',
       'Wind Bearing (in degrees)', 'Visibility (in km)',
       'Pressure (millibars)', 'day', 'month', 'hour'],
      dtype='object')

In [34]:
tdf = pd.read_excel('test_data.xlsx')

In [35]:
y_pred = model.predict(x_test)
from sklearn import metrics

In [36]:
print('R2 :',metrics.r2_score(y_test,y_pred))

R2 : 0.9733275984805294


In [37]:
print('MAE :',metrics.mean_absolute_error(y_test,y_pred))

MAE : 1.2975721347871987


In [38]:
print('RMSE :',np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

RMSE : 1.5695129966606371


In [39]:
tdf.drop(['ID','Day Summary','Precipitation Type'], axis=1, inplace=True)

In [40]:
tdf['Date'] = pd.to_datetime(tdf['Date'],utc=True)
tdf['day'] = tdf['Date'].dt.day
tdf['month'] = tdf['Date'].dt.day
tdf['hour'] = tdf['Date'].dt.hour

In [41]:
tdf

Index(['Date', 'Apparent Temperature (in C)', 'Humidity',
       'Wind Speed (in km/h)', 'Wind Bearing (in degrees)',
       'Visibility (in km)', 'Pressure (millibars)', 'day', 'month', 'hour'],
      dtype='object')

In [42]:
tdf.drop(['Date'], axis=1, inplace=True)

In [43]:
len(dff.columns)

9

In [44]:
len(tdf.columns)

9

In [51]:
sc = StandardScaler()
dff1 = sc.fit_transform(dff)
tdf1 = sc.transform(tdf)

In [52]:
dff1 = dff1.reshape((dff1.shape[0],dff1.shape[1],1))
tdf1 = tdf1.reshape((tdf1.shape[0],tdf1.shape[1],1))

In [53]:
history = model.fit(dff1,y, epochs=5, batch_size=64,validation_data=(dff1,y))

Train on 79857 samples, validate on 79857 samples
Epoch 1/5
79857/79857 [==============================] - 12s 147us/sample - loss: 0.9910 - val_loss: 0.8606
Epoch 2/5
79857/79857 [==============================] - 12s 147us/sample - loss: 0.8986 - val_loss: 0.7184
Epoch 3/5
79857/79857 [==============================] - 12s 145us/sample - loss: 0.8238 - val_loss: 0.5762
Epoch 4/5
79857/79857 [==============================] - 12s 145us/sample - loss: 0.7692 - val_loss: 0.1583
Epoch 5/5
79857/79857 [==============================] - 12s 146us/sample - loss: 0.7237 - val_loss: 0.9395


In [56]:
pred1 = model.predict(tdf1)

In [58]:
pd.DataFrame(pred1).to_csv('Attempt1try.csv')